In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

In [2]:
from desc.grid import ConcentricGrid
import numpy as np

In [3]:
np.set_printoptions(precision=3, floatmode="fixed")
rng = np.random.default_rng()

In [4]:
L = rng.integers(low=1, high=100)
M = rng.integers(low=1, high=100)
N = rng.integers(low=1, high=100)
print(L, M, N)
grid = ConcentricGrid(L=L, N=N, M=M, node_pattern="jacobi")
# print("nodes", "             ", "spacing")
# for a, b in zip(grid.nodes, grid.spacing):
#     print(a, b)

93 18 82


In [5]:
rho = grid.nodes[:, 0]
unique_rho = np.unique(rho)
num = np.random.random_sample(size=len(rho))
den = np.random.random_sample(size=len(rho))
iota_1 = np.zeros(len(unique_rho))
iota_2 = np.zeros(len(unique_rho))

In [6]:
# DESIRED ALGORITHM
# collect collocation indices for each constant rho flux surface
surfaces = dict()
for index, r in enumerate(rho):
    surfaces.setdefault(r, list()).append(index)
# flux surface average integration
for i, surface in enumerate(surfaces.values()):
    iota_1[i] += num[surface].sum()
    iota_1[i] /= den[surface].sum()

print(iota_1)

[1.075 1.061 0.979 1.004 0.981 1.005 0.998 0.960 0.982 0.997 0.992 1.022
 0.961 1.008 0.989 1.028 1.039 1.007 0.980 1.004 1.015 0.988 0.992 1.008
 1.006 0.997 0.978 0.995 0.980 0.972 0.995 0.994 0.986 1.008 0.993 0.999
 1.006 1.014 1.024 1.007 0.996 0.991 1.022 1.011 0.995 1.011 0.991]


In [7]:
# NO LOOP IMPLEMENTATION
bins = np.append(np.unique(rho), 1)
iota_2 += np.histogram(rho, bins=bins, weights=num)[0]
iota_2 /= np.histogram(rho, bins=bins, weights=den)[0]
print(iota_2)

# bincount, bins = np.histogram(rho, bins=bins)
# print(rho)
# print(bincount)
# print(bins)

[1.075 1.061 0.979 1.004 0.981 1.005 0.998 0.960 0.982 0.997 0.992 1.022
 0.961 1.008 0.989 1.028 1.039 1.007 0.980 1.004 1.015 0.988 0.992 1.008
 1.006 0.997 0.978 0.995 0.980 0.972 0.995 0.994 0.986 1.008 0.993 0.999
 1.006 1.014 1.024 1.007 0.996 0.991 1.022 1.011 0.995 1.011 0.991]


In [8]:
assert np.allclose(iota_1, iota_2, equal_nan=True)